In [68]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

In [34]:
'''This is just where I brought in data from an excel spreadsheet.'''
#hourlydata = pd.read_excel('fuelbyhour.xlsx')

In [35]:
'''Remove the year as its always the same'''
hourlydata = hourlydata.drop(['DeliveryYear'],axis=1)

In [ ]:
'''Fills in the hour for the types as it read as NaN.'''
for j in range(len(hourlydata)-1):
    if math.isnan(hourlydata.iloc[j+1,1]) == True:
        hourlydata.iloc[j+1,1] = hourlydata.iloc[j,1]
hourlydata

In [99]:
'''Fills in the date on the entries.'''
Na = hourlydata['Day'].isna()
for j in range(len(hourlydata)-1):
    if Na[j+1] == True:
        hourlydata.iloc[j+1,0] = hourlydata.iloc[j,0]

In [145]:
'''Converts from integer to float because python wants that for some reason.'''
for i in range(len(workingframe)):
    workingframe.iloc[i,2] = float(workingframe.iloc[i,2])

In [165]:
'''Puts the data together nicely as "newframe" '''
newframe = (workingframe.groupby(['Day','Hour','Fuel'])['Output'].mean())

In [168]:
'''Sends the structure to a pickle for use in other scripts'''
newframe.to_pickle('EnergyOutput')

In [169]:
newframe

Day         Hour  Fuel   
2018-01-01  1.0   BIOFUEL        3.000000
                  GAS         1583.000000
                  HYDRO       4152.000000
                  NUCLEAR    11671.000000
                  SOLAR          0.000000
                  WIND         992.000000
            2.0   BIOFUEL        1.000000
                  GAS         1192.000000
                  HYDRO       4093.000000
                  NUCLEAR    11672.000000
                  SOLAR          0.000000
                  WIND        1124.000000
            3.0   BIOFUEL        0.000000
                  GAS         1040.000000
                  HYDRO       4231.000000
                  NUCLEAR    11672.000000
                  SOLAR          0.000000
                  WIND        1265.000000
            4.0   BIOFUEL        0.000000
                  GAS         1041.000000
                  HYDRO       3895.000000
                  NUCLEAR    11669.000000
                  SOLAR          0.000000
        